# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)

In [3]:
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:

train_doc = np.random.permutation(open('/Users/vprasath/Documents/MLAI Course/CNN RNN Case Study/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/Users/vprasath/Documents/MLAI Course/CNN RNN Case Study/Project_data/val.csv').readlines())
batch_size = 2

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = '/Users/vprasath/Documents/MLAI Course/CNN RNN Case Study/Project_data/train'
val_path = '/Users/vprasath/Documents/MLAI Course/CNN RNN Case Study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [6]:
def generator(source_path, folder_list, batch_size):
    #print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = 30
    t = np.random.permutation(folder_list)
    num_batches = get_number_of_batches(len(folder_list),batch_size)
    while True:
        for batch in range(num_batches): # we iterate over the number of batches
            current_batch_size = get_current_batch_size(batch, num_batches, batch_size, len(folder_list))
            batch_data = np.zeros((current_batch_size,img_idx,160,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB 
            batch_labels = np.zeros((current_batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(current_batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*current_batch_size)].split(';')[0]) # read all the images in the folder
                for idx in range(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image_path = source_path+'/'+ t[folder + (batch*current_batch_size)].strip().split(';')[0]+'/'+imgs[idx]
                    batch_data[folder,idx,:,:,0] = resize_and_normalize_image(image_path,120,160,"Red")
                    batch_data[folder,idx,:,:,1] = resize_and_normalize_image(image_path,120,160,"Green")
                    batch_data[folder,idx,:,:,2] = resize_and_normalize_image(image_path,120,160,"Blue")
                batch_labels[folder, int(t[folder + (batch*current_batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember 

In [7]:
# Calculate the number of batches based on the number of sequence available and the batch size

def get_number_of_batches(num_sequences, batch_size):
    if (num_sequences%batch_size) == 0:
        num_batches = int(num_sequences/batch_size)
    else:
        num_batches = (num_sequences//batch_size) + 1
        
    return num_batches

In [8]:
# This function should return the batch size as it is, if it is not the last batch. If it is the last batch calculate
# the remaining number of sequence available and return it. 

def get_current_batch_size(batch, num_batches, batch_size, num_sequences):
    if (batch == (num_batches-1)):
        if (num_sequences%batch_size) == 0:
            current_batch_size = batch_size
        else:
            current_batch_size = int(num_sequences%batch_size)
    else:
        current_batch_size = batch_size
    
    return current_batch_size

In [9]:
# Test get_number_of_batches and get_current_batch_size functions

num_sequences = 58
batch_size = 16

num_batches = get_number_of_batches(num_sequences, batch_size)
print("num_batches: ", num_batches)
for batch in range(num_batches):
    current_batch_size= get_current_batch_size(batch, num_batches,batch_size, num_sequences)
    print("batch: ",batch,"; current_batch_size: ", current_batch_size)
   


num_batches:  4
batch:  0 ; current_batch_size:  16
batch:  1 ; current_batch_size:  16
batch:  2 ; current_batch_size:  16
batch:  3 ; current_batch_size:  10


In [10]:
# Resize and normalize the image 

from PIL import Image
import numpy as np

def resize_and_normalize_image(image_path, width, height, color_channel):
    # Open the image using PIL
    image = Image.open(image_path)
    
    # Resize the image
    resized_image = image.resize((width, height))
    
    # Split the image into individual channels
    red_channel, green_channel, blue_channel = resized_image.split()
    
    # Select the specified color channel
    if color_channel == "Red":
        channel = red_channel
    elif color_channel == "Green":
        channel = green_channel
    elif color_channel == "Blue":
        channel = blue_channel
    else:
        raise ValueError("Invalid color channel. Please choose Red, Green, or Blue.")
    
    # Convert the selected channel to a numpy array
    channel_array = np.array(channel, dtype=np.float32)
    
    # Normalize the channel array
    normalized_channel_array = channel_array / 255.0
    
    return normalized_channel_array

In [11]:
# Test resize_and_normalize_image function

image_path ="/Users/vprasath/Documents/MLAI Course/CNN RNN Case Study/Project_data/train/WIN_20180925_17_17_26_Pro_Right_Swipe_new/WIN_20180925_17_17_26_Pro_00042.png"
color_channel = "Red"  # Choose Red, Green, or Blue
resized_array = resize_and_normalize_image(image_path, 160, 120, color_channel)  # Replace 100 with the desired width and height

# Print the type and shape of the resized image array
print(type(resized_array))
print(resized_array.shape)

<class 'numpy.ndarray'>
(120, 160)


In [12]:
# Test generator function
train_generator = generator(train_path, train_doc, batch_size)
next(train_generator)

(array([[[[[0.3882353 , 0.47058824, 0.44313726],
           [0.38431373, 0.47058824, 0.44313726],
           [0.39215687, 0.47450981, 0.44705883],
           ...,
           [0.53725493, 0.66274512, 0.6156863 ],
           [0.53725493, 0.65882355, 0.60784316],
           [0.54509807, 0.66666669, 0.61960787]],
 
          [[0.3882353 , 0.47058824, 0.44313726],
           [0.38431373, 0.47058824, 0.44313726],
           [0.3882353 , 0.47058824, 0.44313726],
           ...,
           [0.5411765 , 0.66274512, 0.61960787],
           [0.5411765 , 0.66274512, 0.60784316],
           [0.5411765 , 0.66274512, 0.61176473]],
 
          [[0.39607844, 0.47843137, 0.4509804 ],
           [0.39215687, 0.47450981, 0.44705883],
           [0.39215687, 0.47450981, 0.44705883],
           ...,
           [0.5411765 , 0.66274512, 0.61960787],
           [0.5411765 , 0.66274512, 0.6156863 ],
           [0.54509807, 0.66666669, 0.61960787]],
 
          ...,
 
          [[0.        , 0.00392157, 0.      

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

# Model 
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

# Model 1 - 3D CNN

In [15]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras import metrics

# Define your 3D CNN model
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(30, 120, 160, 3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(5, activation='softmax'))

Metal device set to: Apple M1


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [16]:
optimiser = 'adam'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 28, 118, 158, 32)  2624      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 14, 59, 79, 32)   0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 59, 79, 32)    0         
                                                                 
 conv3d_1 (Conv3D)           (None, 12, 57, 77, 64)    55360     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 6, 28, 38, 64)    0         
 3D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 6, 28, 38, 64)     0

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [17]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [27]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [24]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1
if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [26]:
history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/var/folders/93/8119n4wj4xg4l8yw4wgsbhkc0000gn/T/ipykernel_10094/3383535347.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/30
42/42 [==============================] - ETA: 0s - loss: 2.1030 - categorical_accuracy: 0.2081 
Epoch 1: saving model to model_init_2023-05-2309_12_52.068211/model-00001-2.10297-0.20814-1.56329-0.26000.h5
42/42 [==============================] - 579s 14s/step - loss: 2.1030 - categorical_accuracy: 0.2081 - val_loss: 1.5633 - val_categorical_accuracy: 0.2600 - lr: 0.0010
Epoch 2/30
42/42 [==============================] - ETA: 0s - loss: 1.5105 - categorical_accuracy: 0.3379 
Epoch 2: saving model to model_init_2023-05-2309_12_52.068211/model-00002-1.51049-0.33786-1.45533-0.32000.h5
42/42 [==============================] - 497s 12s/step - loss: 1.5105 - categorical_accuracy: 0.3379 - val_loss: 1.4553 - val_categorical_accuracy: 0.3200 - lr: 0.0010
Epoch 3/30
42/42 [==============================] - ETA: 0s - loss: 1.3090 - categorical_accuracy: 0.4615 
Epoch 3: saving model to model_init_2023-05-2309_12_52.068211/model-00003-1.30902-0.46154-1.47954-0.31000.h5
42/42 [=========

42/42 [==============================] - ETA: 0s - loss: 0.0054 - categorical_accuracy: 1.0000 
Epoch 22: saving model to model_init_2023-05-2309_12_52.068211/model-00022-0.00539-1.00000-2.40897-0.41000.h5
42/42 [==============================] - 476s 11s/step - loss: 0.0054 - categorical_accuracy: 1.0000 - val_loss: 2.4090 - val_categorical_accuracy: 0.4100 - lr: 1.0000e-04
Epoch 23/30
42/42 [==============================] - ETA: 0s - loss: 0.0047 - categorical_accuracy: 1.0000 
Epoch 23: saving model to model_init_2023-05-2309_12_52.068211/model-00023-0.00470-1.00000-2.43990-0.41000.h5
42/42 [==============================] - 473s 11s/step - loss: 0.0047 - categorical_accuracy: 1.0000 - val_loss: 2.4399 - val_categorical_accuracy: 0.4100 - lr: 1.0000e-04
Epoch 24/30
42/42 [==============================] - ETA: 0s - loss: 0.0042 - categorical_accuracy: 1.0000 
Epoch 24: saving model to model_init_2023-05-2309_12_52.068211/model-00024-0.00418-1.00000-2.46456-0.41000.h5
42/42 [=======

# Model 2 - CNN + RNN

In [28]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras import metrics

# Define CNN model
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(120, 160, 3)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Flatten())

# Define GRU model
gru_model = Sequential()
gru_model.add(GRU(128, input_shape=(None, cnn_model.output_shape[1])))
gru_model.add(Dense(5, activation='softmax'))


# Combine the CNN and GRU models
combined_model= Sequential()
combined_model.add(TimeDistributed(cnn_model, input_shape=(20, 120, 160, 3)))  # Assuming 10 image frames
combined_model.add(TimeDistributed(Flatten()))
combined_model.add(gru_model)

In [29]:
combined_optimiser = 'adam'
combined_model.compile(optimizer=combined_optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (combined_model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_2 (TimeDis  (None, 20, 14976)        56320     
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 20, 14976)        0         
 tributed)                                                       
                                                                 
 sequential_5 (Sequential)   (None, 5)                 5801349   
                                                                 
Total params: 5,857,669
Trainable params: 5,857,669
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
combined_model_history = combined_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, callbacks=callbacks_list, validation_data=val_generator, validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/var/folders/93/8119n4wj4xg4l8yw4wgsbhkc0000gn/T/ipykernel_10094/2688891503.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  combined_model_history = combined_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, callbacks=callbacks_list, validation_data=val_generator, validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)


Epoch 1/30
42/42 [==============================] - ETA: 0s - loss: 1.8582 - categorical_accuracy: 0.2278
Epoch 1: saving model to model_init_2023-05-2309_12_52.068211/model-00001-1.85820-0.22775-1.54858-0.29000.h5
42/42 [==============================] - 131s 3s/step - loss: 1.8582 - categorical_accuracy: 0.2278 - val_loss: 1.5486 - val_categorical_accuracy: 0.2900 - lr: 0.0010
Epoch 2/30
42/42 [==============================] - ETA: 0s - loss: 1.4191 - categorical_accuracy: 0.4057
Epoch 2: saving model to model_init_2023-05-2309_12_52.068211/model-00002-1.41912-0.40573-1.35547-0.48000.h5
42/42 [==============================] - 129s 3s/step - loss: 1.4191 - categorical_accuracy: 0.4057 - val_loss: 1.3555 - val_categorical_accuracy: 0.4800 - lr: 0.0010
Epoch 3/30
42/42 [==============================] - ETA: 0s - loss: 1.2394 - categorical_accuracy: 0.5173
Epoch 3: saving model to model_init_2023-05-2309_12_52.068211/model-00003-1.23939-0.51735-1.25529-0.50000.h5
42/42 [==============

42/42 [==============================] - ETA: 0s - loss: 0.0115 - categorical_accuracy: 1.0000
Epoch 22: saving model to model_init_2023-05-2309_12_52.068211/model-00022-0.01147-1.00000-1.67610-0.59000.h5
42/42 [==============================] - 130s 3s/step - loss: 0.0115 - categorical_accuracy: 1.0000 - val_loss: 1.6761 - val_categorical_accuracy: 0.5900 - lr: 1.0000e-04
Epoch 23/30
42/42 [==============================] - ETA: 0s - loss: 0.0097 - categorical_accuracy: 1.0000
Epoch 23: saving model to model_init_2023-05-2309_12_52.068211/model-00023-0.00967-1.00000-1.69185-0.59000.h5
42/42 [==============================] - 129s 3s/step - loss: 0.0097 - categorical_accuracy: 1.0000 - val_loss: 1.6919 - val_categorical_accuracy: 0.5900 - lr: 1.0000e-04
Epoch 24/30
42/42 [==============================] - ETA: 0s - loss: 0.0083 - categorical_accuracy: 1.0000
Epoch 24: saving model to model_init_2023-05-2309_12_52.068211/model-00024-0.00825-1.00000-1.70611-0.60000.h5
42/42 [============